# Proximity and Accessibility analysis using Pandana

In this part, we will demonstrate how to perform proximity and accessibility analysis using the `pandana` library. The analysis will be carried out to one city: Delft.

In the next part, we will see how to scale this up to 9 cities.

## Set up environment

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"  # disable Pandana multithreading 
os.environ["USE_PYGEOS"] = "0"  # suppress geopandas warning

import pandana
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt

## Data Loading

We will load data from the following path on Spider: `/project/stursdat/Data/ScalableGIS/Part2/data_9_cities`. We use the `pathlib.Path` library to handle the path object.

In [2]:
from pathlib import Path

data_folder = Path("data")

In the `data_9_cities` folder, there are 9 folders for 9 Dutch cities. Within each city folder, there are `.shp` files for four componets of that city:

- **buildings**: building polygons
- **parks**: park polygons
- **nodes**: street network nodes
- **edges**: street network edges


In [3]:
# Check city folders in the data directory
[d for d in data_folder.glob('*')]

[PosixPath('data/Arnhem'),
 PosixPath('data/Amersfoort'),
 PosixPath('data/Nijmegen'),
 PosixPath('data/Breda'),
 PosixPath('data/Enschede'),
 PosixPath('data/Delft'),
 PosixPath('data/Leiden'),
 PosixPath('data/Gouda'),
 PosixPath('data/Deventer')]

In [4]:
# Check .shp files in Delft
[d for d in data_folder.glob('Delft/*.shp')]

[PosixPath('data/Delft/parks_Delft.shp'),
 PosixPath('data/Delft/edges_Delft.shp'),
 PosixPath('data/Delft/buildings_Delft.shp'),
 PosixPath('data/Delft/nodes_Delft.shp')]

In [5]:
# For now we focus on the city Delft
city = 'Delft'

In [6]:
# Paths to the files
p_nodes = data_folder / city / "nodes_{}.shp".format(city)
p_edges = data_folder / city / "edges_{}.shp".format(city)
p_parks = data_folder / city / "parks_{}.shp".format(city)
p_buildings = data_folder / city / "buildings_{}.shp".format(city)

In [7]:
# Check if paths exist
for p in [p_nodes, p_edges, p_parks, p_buildings]:
    print(p.exists())

True
True
True
True


In [9]:
# load data with GeoPandas
nodes = gpd.read_file(p_nodes)
edges = gpd.read_file(p_edges)
parks = gpd.read_file(p_parks)
buildings = gpd.read_file(p_buildings)

In [11]:
edges

,u,v,key,length,geometry
0,25315544,493868196,0,30.991,"LINESTRING (86125.744 448869.147, 86106.344 44..."
1,25316215,1432937725,0,87.869,"LINESTRING (85821.048 448466.589, 85827.203 44..."
2,25316229,1271057030,0,206.947,"LINESTRING (86299.010 448536.557, 86295.641 44..."
3,25316229,302097504,0,25.239,"LINESTRING (86299.010 448536.557, 86304.549 44..."
4,25316229,302095241,0,25.849,"LINESTRING (86299.010 448536.557, 86291.946 44..."
...,...,...,...,...,...
26747,10885892864,9733120577,0,115.073,"LINESTRING (83422.409 445125.064, 83421.421 44..."
26748,10885892891,10885892892,0,7.328,"LINESTRING (83431.408 444929.117, 83424.056 44..."
26749,10885892892,9733120579,0,23.696,"LINESTRING (83424.056 444929.078, 83423.544 44..."
26750,10885892892,9733120575,0,16.168,"LINESTRING (83424.056 444929.078, 83424.336 44..."


In [12]:
# Set indexes for nodes and edges
nodes = nodes.set_index("osmid", drop=False)
edges = edges.set_index(["u", "v"], drop=False)

In [13]:
# setup a network
network = pandana.Network(
    node_x=nodes["x"], 
    node_y=nodes["y"], 
    edge_from=edges["u"], 
    edge_to=edges["v"], 
    edge_weights=edges[["length"]],
)

Generating contraction hierarchies with 1 threads.
Setting CH node vector of size 9762
Setting CH edge vector of size 27352
Range graph removed 28274 edges of 54704
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

In [23]:
network

## Proximity analysis 

In [ ]:
# add point of interests to network
network.set_pois(
    category="parks",
    maxdist=1000,
    maxitems=25,
    x_col=parks.centroid.x,
    y_col=parks.centroid.y,
)

In [ ]:
# for all nodes, find 3 closest parks within 800m
results = network.nearest_pois(
    distance=800,
    category="parks",
    num_pois=3,
    include_poi_ids=False
)

In [ ]:
results.describe()

## Accessibility analysis

In [ ]:
# add target points to network
node_ids = network.get_node_ids(parks.centroid.x, y_col=parks.centroid.y)
network.set(node_ids, name="parks")

In [ ]:
# for all nodes, find how many parks fall within 800m
accessibility = network.aggregate(
    distance=800,
    type="count",
    name="parks"
)

In [ ]:
# visualize nodes using accessibility
fig, ax = plt.subplots(1, 1, figsize=(10,8))
plt.title('Delft: Parks within 800m')
parks.plot(
    ax=ax,
    color="green"
)
plt.scatter(
    nodes["x"], 
    nodes["y"], 
    c=accessibility, 
    s=1, 
    cmap='autumn', 
    norm=matplotlib.colors.LogNorm()
)
cb = plt.colorbar()

In [ ]:
# assign accessibility of buildings using closest nodes 
node_ids = network.get_node_ids(
    x_col=buildings.centroid.x, 
    y_col=buildings.centroid.y
)
buildings["accessibility"] = node_ids.map(accessibility.to_dict())

In [ ]:
# visualize buildings using accessibility
fig, ax = plt.subplots(1, 1, figsize=(10,8))
plt.title('Delft: Parks within 800m')
parks.plot(
    ax=ax,
    color="green"
)
buildings.plot(
    ax=ax,
    column="accessibility",  
    cmap="autumn",
    norm=matplotlib.colors.LogNorm(),
    figsize=(10,8),
)